In [19]:
# https://python.langchain.com/docs/integrations/vectorstores/opensearch

In [1]:
!pip install opensearch-py

  Obtaining dependency information for opensearch-py from https://files.pythonhosted.org/packages/6a/88/8f48281e1f8d48746ab7061a7c2ab37b4a57b358f04159cd9daa3dafe04b/opensearch_py-2.4.2-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.6/258.6 kB 16.8 MB/s eta 0:00:00


In [11]:
!pip install requests-aws4auth
!pip install boto3

In [1]:
import getpass
import os

#os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [2]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import OpenSearchVectorSearch

In [3]:
from langchain.document_loaders import TextLoader

loader = TextLoader("../data/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# OpenAIEmbeddings 사용시 SSLCertVerificationError 발생
#embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [4]:
from requests_aws4auth import AWS4Auth
import boto3
from opensearchpy import RequestsHttpConnection

In [5]:
# This is just an example to show how to use Amazon OpenSearch Service, you need to set proper values.
import boto3
from opensearchpy import RequestsHttpConnection

service = "es"  # must set the service as 'es'
region = "ap-northeast-2"

In [6]:
credentials = boto3.Session(
    aws_access_key_id="----", aws_secret_access_key="----"
).get_credentials()
awsauth = AWS4Auth("----", "----", region, service, session_token=credentials.token)

session = boto3.Session()
credentials = session.get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, 'es', session_token=credentials.token)

In [7]:
from opensearchpy import OpenSearch, Urllib3HttpConnection, Urllib3AWSV4SignerAuth

credentials = boto3.Session().get_credentials()
auth = Urllib3AWSV4SignerAuth(credentials, region, service)

host = 'vpc-dev-search-es-250-crdyurhlzxphmrjdfmtz7m23nu.ap-northeast-2.es.amazonaws.com'

client = OpenSearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = Urllib3HttpConnection,
    pool_maxsize = 20
)

In [8]:
info = client.info()
print(f"Welcome to {info}!")
#print(f"Welcome to {info['version']['distribution']} {info['version']['number']}!")

Welcome to {'name': '07d3e7c3ade78c4a51db185ae66c04b6', 'cluster_name': '175979101058:dev-search-es-250', 'cluster_uuid': 'uPQyc5N1Ri-lw1OxbsjxaQ', 'version': {'number': '7.10.2', 'build_type': 'tar', 'build_hash': 'unknown', 'build_date': '2023-04-20T10:37:24.437846Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.10.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'The OpenSearch Project: https://opensearch.org/'}!


In [12]:
docsearch = OpenSearchVectorSearch.from_documents(
    docs,
    embeddings,
    opensearch_url="https://vpc-dev-search-es-250-crdyurhlzxphmrjdfmtz7m23nu.ap-northeast-2.es.amazonaws.com",
    http_auth=auth,
    timeout=300,
    use_ssl=True,
    verify_certs=True,
    connection_class=Urllib3HttpConnection,
    index_name="test-vector"
)

In [18]:
query = "What did the president say about Ketanji Brown Jackson"
docs = docsearch.similarity_search(query, k=10)

In [19]:
print(docs)

[Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': '../data/state_of_the_union.txt'}), Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act

In [9]:
#from opensearch_vector_search import OpenSearchVectorSearch2

In [10]:
docsearch = OpenSearchVectorSearch(
    opensearch_url="https://vpc-dev-search-es-250-crdyurhlzxphmrjdfmtz7m23nu.ap-northeast-2.es.amazonaws.com",
    embedding_function=embeddings,
    http_auth=auth,
    timeout=300,
    use_ssl=True,
    verify_certs=True,
    connection_class=Urllib3HttpConnection,
    index_name="test-vector",
)

In [16]:
docs = docsearch.similarity_search(
    "Who was asking about getting lunch today?",
#    search_type="script_scoring",
#    space_type="cosinesimil",
#    vector_field="message_embedding",
#    text_field="message",
#    metadata_field="message_metadata",
    k=4
)

In [17]:
print(docs)

[Document(page_content='And my report is this: the State of the Union is strong—because you, the American people, are strong. \n\nWe are stronger today than we were a year ago. \n\nAnd we will be stronger a year from now than we are today. \n\nNow is our moment to meet and overcome the challenges of our time. \n\nAnd we will, as one people. \n\nOne America. \n\nThe United States of America. \n\nMay God bless you all. May God protect our troops.', metadata={'source': '../data/state_of_the_union.txt'}), Document(page_content='And my report is this: the State of the Union is strong—because you, the American people, are strong. \n\nWe are stronger today than we were a year ago. \n\nAnd we will be stronger a year from now than we are today. \n\nNow is our moment to meet and overcome the challenges of our time. \n\nAnd we will, as one people. \n\nOne America. \n\nThe United States of America. \n\nMay God bless you all. May God protect our troops.', metadata={'source': '../data/state_of_the_u